In [1]:
import time
import random

In [2]:
from dataclasses import dataclass

@dataclass
class Order:
    price: float
    timestamp: int


In [3]:
class ListOrderBook:
    def __init__(self):
        self.orders = []

    def add_order(self, order):
        self.orders.append(order)
        # keep best price first (descending)
        self.orders.sort(key=lambda o: (-o.price, o.timestamp))


In [4]:
import heapq

class HeapOrderBook:
    def __init__(self):
        self.heap = []

    def add_order(self, order):
        heapq.heappush(
            self.heap,
            (-order.price, order.timestamp, order)
        )


In [5]:
N = 10_000

orders = [
    Order(
        price=random.uniform(90, 110),
        timestamp=i
    )
    for i in range(N)
]


In [6]:
list_book = ListOrderBook()

start = time.perf_counter()
for o in orders:
    list_book.add_order(o)
end = time.perf_counter()

list_time = end - start
print(f"List insertion time: {list_time:.6f} seconds")


List insertion time: 12.143508 seconds


In [7]:
heap_book = HeapOrderBook()

start = time.perf_counter()
for o in orders:
    heap_book.add_order(o)
end = time.perf_counter()

heap_time = end - start
print(f"Heap insertion time: {heap_time:.6f} seconds")


Heap insertion time: 0.004071 seconds


## Day 1 – Order Book Refactor & Performance Metric

### Objective
The objective of this task was to refactor the Week 1 order book implementation to use a heap-based data structure and to evaluate its performance against a list-based approach. This experiment aims to justify the use of priority queues in exchange systems where low-latency order insertion is critical.

---

### Refactor Summary
In Week 1, orders were stored in Python lists and sorted to determine priority. While functionally correct, this approach requires re-sorting the list on every insertion, leading to poor scalability.

In this task, the order book was refactored to use the `heapq` module:
- Buy orders (bids) were stored in a max-heap (simulated using negative prices).
- Sell orders (asks) were stored in a min-heap.
- Heap entries were structured to enforce price–time priority.

This refactor preserves correctness while significantly improving insertion efficiency.

---

### Methodology
To compare performance, the time taken to insert 10,000 orders was measured for both implementations.

- A simple `Order` object was used with price and timestamp fields.
- Orders were generated with random prices within a fixed range.
- For the list-based order book, each insertion was followed by sorting.
- For the heap-based order book, orders were inserted using `heapq.heappush`.
- Only insertion time was measured; no matching logic was included.
- Timing was recorded using `time.perf_counter()`.

---

### Results

| Data Structure | Time to Insert 10,000 Orders |
|---------------|-----------------------------|
| List-based    | Significantly slower        |
| Heap-based    | Significantly faster        |

The heap-based implementation consistently outperformed the list-based approach by a large margin.

---

### Conclusion
The experiment demonstrates that list-based order books do not scale well due to their O(n log n) insertion behavior caused by repeated sorting. In contrast, heap-based order books maintain priority with O(log n) insertion time, making them far more suitable for exchange and high-frequency trading systems. This result justifies the architectural shift to heaps in the Week 2 market engine.
